# SNAP Databases

* Author: Xin Chen
* Email: Bismarrck@me.com
* Date: Jan 10, 2019
* Reference: [PHYSICAL REVIEW B 98, 094104 (2018)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.98.094104)

This notebook demonstrates how to build 
**[SNAP](http://pymatgen.org/_modules/pymatgen/core/structure.html#IMolecule.as_dict)** databases for **TensorAlloy**.
The SNAP databases have three sets:

1. Ni-Mo: 3973 structures
2. Ni: 461 structures
3. Mo: 284 structures

In [1]:
from __future__ import print_function

In [10]:
import zipfile
import shutil
import json
import glob
import os
import requests

from os.path import join, basename, splitext, exists
from pymatgen.core.structure import Structure
from ase.calculators.singlepoint import SinglePointCalculator
from ase import Atoms
from ase.io import write, read
from typing import List

Below is the function to convert a list of `dict` to `ase.Atoms` objects and then
write the trajectory to an `extxyz` files.

**Note:** the unit of the stress values is `-1  * kbar`.

In [3]:
def convert_pymatgen_json_to_extxyz(list_of_dicts: List[dict], output_file):
    """
    Convert the JSON-serialized structures in `list_of_dicts` to `Atoms` 
    objects and write the trajectory to an `extxyz` file.
    
    Parameters
    ----------
    list_of_dicts : List[dict]
        A list of JSON-serialized dicts.
    output_file : str
        The output file to write.
    
    Returns
    -------
    ntotal : int
        The total number of structures found.
    
    """
    trajectory = []
    for i in range(len(list_of_dicts)):
        structure = Structure.from_dict(list_of_dicts[i]['structure'])
        atoms = Atoms(list(map(str, structure.species)), 
                      structure.cart_coords, 
                      cell=structure.lattice.matrix,
                      pbc=[True, True, True])
        if 'outputs' in list_of_dicts[i]:
            outputs = list_of_dicts[i]['outputs']
            outputs['stress'] = [-x for x in outputs['stress']]
            calc = SinglePointCalculator(atoms, **outputs)
            atoms.calc = calc
            params = list_of_dicts[i]['params']
            params.pop('pp')
            atoms.info = dict(tags=list_of_dicts[i]['tags'], **params)
        else:
            # The Mo dataset was created separately so the format is different.
            data = list_of_dicts[i]['data']
            results = {
                'energy': data['energy_per_atom'] * len(atoms),
                'forces': data['forces'],
                'stress': [-x for x in data['virial_stress']],
            }
            calc = SinglePointCalculator(atoms, **results)
            atoms.calc = calc
        trajectory.append(atoms)
    write(output_file, trajectory, format='extxyz')
    return len(trajectory)

Download the original SNAP dataset if needed. Then unzip this file.

In [12]:
local_dir = 'snap-master'
local_filename = 'snap-master.zip'

def download_or_unzip():
    if not exists(local_filename):
        url = "https://codeload.github.com/materialsvirtuallab/snap/zip/master"
        r = requests.get(url, stream=True)
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    if exists(local_dir):
        shutil.rmtree(local_dir, ignore_errors=True)
    with zipfile.ZipFile(local_filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    if exists('__MACOSX'):
        shutil.rmtree('__MACOSX')

download_or_unzip()

Generate the extxyz files.

There are five types of subsets:

* Cu : aimd, elastic, md, surface, vacancy
* Mo : aimd (npt), aimd (nvt), elastic, gb, surface, vacancy, doped_with_ni
* Ni : aimd, elastic, surface, vacancy, surface (test), doped_with_Mo
* Ni3Mo : aimd, elastic
* Ni4Mo : aimd, elastic

In [26]:
sources = ['Mo', 'Ni', 'Ni-Mo']
ntotal = 0

for index, source in enumerate(sources):
    training_dir = join(local_dir, source, 'training')
    list_of_files = list(glob.glob(f"{training_dir}/*.json"))
    for afile in list_of_files:
        if index == 3:
            output_file = f"{splitext(basename(afile))[0]}.extxyz"
        else:
            tag = splitext(basename(afile))[0]
            output_file = f"{source.replace('-', '')}.{tag}.extxyz"
        with open(afile) as fp:
            try:
                n = convert_pymatgen_json_to_extxyz(json.load(fp), output_file)
                ntotal += n
            except Exception as excp:
                print(f"Failed to proceed {afile}: {str(excp)}")
print(f"Total {ntotal} structures.")

Total 3973 structures.


Merge these extxyz files to a single `extxyz` file.

In [27]:
if exists('snap.extxyz'):
    os.remove('snap.extxyz')
all_files = glob.glob('*.*.extxyz')
with open('snap.extxyz', 'w') as fp:    
    for afile in all_files:
        print(afile)
        with open(afile, 'r') as obj:
            fp.write(obj.read())

NiMo.Ni3Mo_AIMD.extxyz
NiMo.Ni4Mo_Elastic.extxyz
Mo.AIMD_NVT.extxyz
Mo.Elastic.extxyz
NiMo.Ni4Mo_AIMD.extxyz
Mo.GB.extxyz
Ni.Surface_test.extxyz
Mo.Vacancy.extxyz
NiMo.Ni_dopedwith_Mo.extxyz
Ni.Surface.extxyz
NiMo.Mo_dopedwith_Ni.extxyz
Ni.AIMD.extxyz
Ni.Elastic.extxyz
Mo.AIMD_NPT.extxyz
Ni.Vacancy.extxyz
NiMo.Ni3Mo_Elastic.extxyz
Mo.Surface.extxyz


Create a **Ni** `extxyz` file.

In [28]:
all_Ni_files = glob.glob('Ni.*.extxyz')
with open('snap-Ni.extxyz', 'w') as fp:    
    for afile in all_Ni_files:
        print(afile)
        with open(afile, 'r') as obj:
            fp.write(obj.read())

Ni.Surface_test.extxyz
Ni.Surface.extxyz
Ni.AIMD.extxyz
Ni.Elastic.extxyz
Ni.Vacancy.extxyz


Create a **Mo** `extxyz` file.

In [29]:
all_Mo_files = glob.glob('Mo.*.extxyz')
with open('snap-Mo.extxyz', 'w') as fp:    
    for afile in all_Mo_files:
        print(afile)
        with open(afile, 'r') as obj:
            fp.write(obj.read())

Mo.AIMD_NVT.extxyz
Mo.Elastic.extxyz
Mo.GB.extxyz
Mo.Vacancy.extxyz
Mo.AIMD_NPT.extxyz
Mo.Surface.extxyz


Delete unnecessary files.

In [32]:
for afile in all_files:
    if exists(afile):
        os.remove(afile)
if exists(local_dir):
    shutil.rmtree(local_dir, ignore_errors=True)